Rag Langchain을 이용해서 한글문서를 기반으로 대화 맥락을 유지하면서 문서를 참조하는 챗봇
```
RAG : Retrieval Argumented Generation
외부문서를 검색(Retrieval) 해서 LLM의 답변생성(Generation)을 강화하는 기술
  문서검색 : 질문과 관련된 문서를 벡터 데이터베이스에서 검색
  문서임베딩 : 텍스트를 벡터화(의미적 유사성)
  답변생성 : 검색된 문서를 LLM전달해서 맥락에 맞는 답변 생성
  장점 :
  한글고려사항  
LangChain : 오픈소스 프레임워크
  Prompt Templates : 동적 프롬프트 생성
  Memory : 대화맥락 유지
  Chains : 작업흐름관리
  Retryival : 외부문서 검색
  한글지원 잘됨, open api와 호환
  문서로드 : TextLoader -> Document객체 변환 - 이후 문서를 작은 단위로 분할(청크)

```

In [ ]:
api_key = 'your_openai_api_key'

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
# 문서 로드
loader = TextLoader('/content/korea_culture.txt')
documents = loader.load()
len(documents)

1

In [ ]:
# 문서 분할
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': '/content/korea_culture.txt'}, page_content='대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.\n한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.\n한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.\n한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.\n한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.'),
 Document(metadata={'source': '/content/korea_culture.txt'}, page_content='한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.\n한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.')]

In [ ]:
# 문서의 내용 및 길이 확인
with open('/content/korea_culture.txt', encoding='utf-8') as f:
  text = f.read()
  print(f'문서길이 : {len(text)}')
  temp = text.count('\n')
  print(f"줄바꿈수' : {temp}")
  print(f"구분점 수 : {text.count('.')}")

문서길이 : 294
줄바꿈수' : 6
구분점 수 : 7


In [ ]:
def load_and_seperare_document(texts = 'korea_culture.txt',chunk_size=100,chunk_overlap=10):
  # 문서로드
  loader = TextLoader(texts)
  documents = loader.load()
  # 문서내용 및 길이 확인
  with open(texts, encoding='utf-8') as f:
    text = f.read()
    print(f'문서길이 : {len(text)}')
    temp = text.count('\n')
    print(f"줄바꿈 수 : {temp}")
    print(f"구분점 수 : {text.count('.')}")
  # 문서 분할
  # 분할된 청크를 반환
  text_splitter =  RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
  texts = text_splitter.split_documents(documents)
  print(f'분할된 청크수 : {len(texts)}')
  for i,chunk in enumerate(texts):
    print(f'청크{i+1} : {chunk.page_content}')
  return texts

In [ ]:
load_and_seperare_document()

문서길이 : 294
줄바꿈 수 : 6
구분점 수 : 7
분할된 청크수 : 4
청크1 : 대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.
한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.
청크2 : 한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.
한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
청크3 : 한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.
한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.
청크4 : 한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.


[Document(metadata={'source': 'korea_culture.txt'}, page_content='대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.\n한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.'),
 Document(metadata={'source': 'korea_culture.txt'}, page_content='한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.'),
 Document(metadata={'source': 'korea_culture.txt'}, page_content='한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.\n한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.'),
 Document(metadata={'source': 'korea_culture.txt'}, page_content='한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.')]

RAG 구현
```
  벡터데이터베이스 Chroma 가볍고 빠름 -> Langchain과 잘어울림
  임베딩 : OpenAIEmbedding : 다국어 지원
  검색 : 질문도 벡터화해서 데이터베이스에 저장된 벡트와 유사도 검색

  RetrivalQA체인:
    질문 -> 벡터데이터베이스 에서 문서 검색 -> 검색된 문서를 LLM에 전달 -> 답변
    
```

In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.1 MB/s eta 0

In [ ]:
! pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
# LLM 초기화
llm = OpenAI(openai_api_key=api_key)
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
texts = load_and_seperare_document()  # 청크분할
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db')
# vectorstore.persist()  # 벡터 데이터 베이스 생성 완료  ince Chroma 0.4.x 이후에서는 자동으로 DB가 저장된다

문서길이 : 294
줄바꿈 수 : 6
구분점 수 : 7
분할된 청크수 : 4
청크1 : 대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.
한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.
청크2 : 한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.
한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
청크3 : 한국은 K-팝과 드라마로 전 세계적으로 문화적 영향력을 확대하고 있습니다.
한국의 주요 명절로는 설날과 추석이 있으며, 가족들이 모여 전통 음식을 나눕니다.
청크4 : 한국의 전통 의상인 한복은 색상과 디자인이 화려하며, 명절이나 결혼식에서 자주 입습니다.


In [ ]:
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':2}),  # 상위 2개 문서 검색
    return_source_documents = True,
)

In [ ]:
# 질문 생성
query = "한국의 공식 언어는 무엇입니까?"
result = qa_chain({'query':query})
# 결과 출력
print(f'질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {doc.page_content for doc in result["source_documents"]}')

<ipython-input-13-f7c8987c25d1>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({'query':query})


질문 : 한국의 공식 언어는 무엇입니까?
답변 :  한글.
참조 문서 : <generator object <genexpr> at 0x784de9b00110>


In [ ]:
# 질문 생성
query = "미국의 공식 언어는 무엇입니까?"
result = qa_chain({'query':query})
# 결과 출력
print(f'질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

질문 : 미국의 공식 언어는 무엇입니까?
답변 :  미국의 공식 언어는 영어입니다.
참조 문서 : ['한국의 공식 언어는 한글로, 세종대왕이 1443년에 창제했습니다.\n한국의 전통 음식으로는 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.', '대한민국은 동아시아에 위치한 나라로, 공식 명칭은 대한민국입니다.\n한국의 수도는 서울이며, 서울은 약 970만 명의 인구를 가진 대도시입니다.']


In [ ]:
# 단순 llm을 사용
llm.invoke(query)

'\n\n미국의 공식 언어는 영어입니다.'

1. 문서선택(text, pdf ,doc 등 다양한 문서 검색 기능을 지원)
2. 문서를 기반으로 벡터데이터 베이스 생성 - RAG
3. RAG를 기반으로 랭체인 구축(RAG를 통해 검색을 해서 해당 데이터를 LLM전달 랭체인)


In [ ]:
# 회사 규정
company_rules = '''
본 회사는 수습기간 4개월 후 평가를 거쳐 정규직 전환이 결정됨
본 회사의 출퇴근 시간은 오전8시 출근 5시 퇴근을 기준으로 함.
전사가 금욜일은 오전 근무만 함
재택근무시 필요에 따라 본인에 판단하에 자유롭게 실시한다
'''
with open('company_rules.txt', 'w') as f:
  f.write(company_rules)

# LLM 초기화
llm = OpenAI(openai_api_key=api_key)
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
texts = load_and_seperare_document('company_rules.txt')  # 청크분할
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db2')
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':2}),  # 상위 2개 문서 검색
    return_source_documents = True,
)
# 질문 생성
query = "우리회사의 재택근무는 어떻게 되나요?"
result = qa_chain({'query':query})
# 결과 출력
print(f'질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

문서길이 : 124
줄바꿈 수 : 5
구분점 수 : 1
분할된 청크수 : 2
청크1 : 본 회사는 수습기간 4개월 후 평가를 거쳐 정규직 전환이 결정됨
본 회사의 출퇴근 시간은 오전8시 출근 5시 퇴근을 기준으로 함.
전사가 금욜일은 오전 근무만 함
청크2 : 재택근무시 필요에 따라 본인에 판단하에 자유롭게 실시한다
질문 : 우리회사의 재택근무는 어떻게 되나요?
답변 :  저는 이 문장에서 재택근무에 대한 언급을 찾지 못했습니다. 즉, 이 문장에서는 재택근무에 대한 정보가 제공되지 않았기 때문에 알 수 없습니다.
참조 문서 : ['본 회사는 수습기간 4개월 후 평가를 거쳐 정규직 전환이 결정됨\n본 회사의 출퇴근 시간은 오전8시 출근 5시 퇴근을 기준으로 함.\n전사가 금욜일은 오전 근무만 함', '본 회사는 수습기간 4개월 후 평가를 거쳐 정규직 전환이 결정됨\n본 회사의 출퇴근 시간은 오전8시 출근 5시 퇴근을 기준으로 함.\n전사가 금욜일은 오전 근무만 함']


In [ ]:
# 청크사이즈, 참조문서의 개수 --> 파이퍼 파라메터
!pip install -U langchain-community -q
!pip install chromadb -q

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import shutil

In [ ]:
def load_and_seperare_document(texts = 'korea_culture.txt',chunk_size=100,chunk_overlap=10):
  # 문서로드
  loader = TextLoader(texts)
  documents = loader.load()
  # 문서내용 및 길이 확인
  with open(texts, encoding='utf-8') as f:
    text = f.read()
    print(f'문서길이 : {len(text)}')
    temp = text.count('\n')
    print(f"줄바꿈 수 : {temp}")
    print(f"구분점 수 : {text.count('.')}")
  # 문서 분할
  # 분할된 청크를 반환
  text_splitter =  RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                separators=['\n\n', '\n', '.', '!', '?', ',', ' '],
                                                keep_separator=True)
  texts = text_splitter.split_documents(documents)
  print(f'분할된 청크수 : {len(texts)}')
  for i,chunk in enumerate(texts):
    print(f'청크{i+1} : {chunk.page_content}')
  return texts

In [ ]:
# 회사규정
company_rules = '''
우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다
'''
with open('company_rules.txt', 'w') as f:
  f.write(company_rules)

# LLM 초기화
llm = OpenAI(openai_api_key=api_key)
# 벡터 데이터베이스 생성
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
texts = load_and_seperare_document('company_rules.txt',chunk_size=50)  # 청크분할
vectorstore =  Chroma.from_documents(texts, embeddings, persist_directory='./db001')
# RetrivalQA 체인 생성
qa_chain =  RetrievalQA.from_chain_type(
    llm = llm,
    # chain_type = 'stuff', # 작은 문서는 분할하지않고 한번에 전달
    retriever = vectorstore.as_retriever(search_kwargs = {'k':5}),  # 상위 2개 문서 검색
    return_source_documents = True,
)
# 질문 생성
query = "우리회사의 근태 규정은 어떻게 되나요?"
result = qa_chain.invoke({'query':query})
# 결과 출력
print(f'\n\n질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

문서길이 : 122
줄바꿈 수 : 5
구분점 수 : 1
분할된 청크수 : 3
청크1 : 우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.
청크2 : 근태관리 기준은 8출근 5퇴근을 기준으로 한다
금요일은 전사가 오전 근무만 한다
청크3 : 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다


질문 : 우리회사의 근태 규정은 어떻게 되나요?
답변 :  근태관리 기준은 8출근 5퇴근을 기준으로 하며, 금요일은 전사가 오전 근무만 합니다. 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시합니다.
참조 문서 : ['우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.', '우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.', '근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다', '근태관리 기준은 8출근 5퇴근을 기준으로 한다\n금요일은 전사가 오전 근무만 한다', '재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다']


Langchain으로 다행한 포메 ㅅ풀어괴

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.2 MB/s eta 0:00:00


In [ ]:
# 텍스트 파일
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("/content/company_rules.pdf")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages

[Document(metadata={'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-12T11:27:49+09:00', 'source': '/content/company_rules.pdf', 'file_path': '/content/company_rules.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '이규영', 'subject': '', 'keywords': '', 'moddate': '2025-05-12T11:27:49+09:00', 'trapped': '', 'modDate': "D:20250512112749+09'00'", 'creationDate': "D:20250512112749+09'00'", 'page': 0}, page_content='우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다. \n근태관리 기준은 8출근 5퇴근을 기준으로 한다 \n금요일은 전사가 오전 근무만 한다 \n우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다')]

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=6e5683d6d77cb7a8e89abc77495ed4cf402e19ba343d44ee2ec30ad2235ab153
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
# 워드문서 로드
from langchain.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader("/content/company_rules.docx")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages


[Document(metadata={'source': '/content/company_rules.docx'}, page_content='우리 회사의 정규직 전환은 수습기간 4개월후 평가를 거쳐 결정된다.\n\n근태관리 기준은 8출근 5퇴근을 기준으로 한다\n\n금요일은 전사가 오전 근무만 한다\n\n우리회사의 재택근무는 본인이 원하면 누구의 승인도 받지 않고 자유롭게 실시한다\n\n구분 종류 타입 가격 사무용 개인용품 A타입 5000')]

In [ ]:
import pandas as pd
data = {
    'name' : ['Alice', 'Bob', 'Charlie'],
    'age' : [25, 30, 35],
    'city' : ['New York', 'San Francisco', 'Los Angeles']
}
df = pd.DataFrame(data)
df.to_csv('example.csv', index=False)
df

,name,age,city
0,Alice,25,New York
1,Bob,30,San Francisco
2,Charlie,35,Los Angeles


In [ ]:
# csv 로드
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("example.csv")
pages = loader.load()
pages


[Document(metadata={'source': 'example.csv', 'row': 0}, page_content='name: Alice\nage: 25\ncity: New York'),
 Document(metadata={'source': 'example.csv', 'row': 1}, page_content='name: Bob\nage: 30\ncity: San Francisco'),
 Document(metadata={'source': 'example.csv', 'row': 2}, page_content='name: Charlie\nage: 35\ncity: Los Angeles')]

In [ ]:
# 웹페이지 로드
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://github.com/sknetworks20250226/AI")
pages = loader.load()
pages

[Document(metadata={'source': 'https://github.com/sknetworks20250226/AI', 'title': 'GitHub - sknetworks20250226/AI', 'description': 'Contribute to sknetworks20250226/AI development by creating an account on GitHub.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub - sknetworks20250226/AI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nGitHub Advanced Security\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev envir

# RAG기반 FAQ구현

In [ ]:
!pip install chromadb -q

In [ ]:
!pip install -U langchain-community -q
!pip install langchain_openai -q

In [ ]:
# 필수 라이브러리
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
api_key = api_key

In [ ]:
!pip install pymupdf

In [ ]:
# 문서로드 및 분할
def load_and_split_text(file_path):
  if file_path.endswith('.pdf'):
    loader = PyMuPDFLoader(file_path)
  else:
    loader = TextLoader(file_path)

  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=90, chunk_overlap=5,
                    separators=['.','\n',' ',''],keep_separator=True       )
  return text_splitter.split_documents(documents)

In [ ]:
# RAG 챗봇구현
def create_rag_chatbot(texts):
  # 벡터 데이터베이스 생성
  vectorstore = Chroma.from_documents(texts, OpenAIEmbeddings(api_key=api_key), persist_directory='./db2')
  # LLm 준비
  llm = OpenAI(temperature=0.2, openai_api_key=api_key, max_tokens=2048)
  # 프롬프트 엔지니어링(한글 답변 최적화)
  prompt_template = PromptTemplate(
      input_variables=['context', 'question'],
      template='''문서 : {context}
      질문:{question}
      다음 단계을 따라 한글로 답변하세요
      1. 문서에서 질문과 관련된 문장 검색.
      2. 문장이 있으면 문장기반으로 답변구성
      3. 문장이 없으면 "문서에 정보 없음"을 명시하고, 일반지식으로 간결히답변
      답변은 정확해야 합니다.
      '''

  )
  # RetrivalQA 체인
  rag_chatbot = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
      chain_type_kwargs={'prompt': prompt_template},
      return_source_documents = True,
  )
  return rag_chatbot

In [ ]:
# 문서로드 및 분할
texts = load_and_split_text("/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf")
print("문서개수:",len(texts) )
qa_chain = create_rag_chatbot(texts)

문서개수: 631


In [ ]:
query = '위험평가의 실시'
result = qa_chain.invoke(query)
print(f'\n\n질문 : {query}')
print(f'답변 : {result["result"]}')
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')



질문 : 위험평가의 실시
답변 : 
위험평가위원회는 위험을 평가하는 기관으로, 위험성을 파악하고 관리하기 위해 설립되었습니다. 따라서 위험평가위원회는 위험평가를 실시하는 역할을 담당합니다. 위험평가는 위험요소를 식별하고 분석하여 위험의 정도를 판단하는 과정을 말합니다. 이를 통해 위험을 최소화하고 안전한 환경을 조성하기 위한 대책을 마련할 수 있습니다. 따라서 위험평가위원회는 위험평가를 실시하는 것이 중요한 역할이며, 이를 통해 사회적 안전을 보장하는 역할을 수행합니다.
참조 문서 : ['(이하 "위험평가위원회"라 한다)를 둔다', '(이하 "위험평가위원회"라 한다)를 둔다', '(이하 "위험평가위원회"라 한다)를 둔다']


In [ ]:
print(result['result'])


위험평가위원회는 위험을 평가하는 기관으로, 위험성을 파악하고 관리하기 위해 설립되었습니다. 따라서 위험평가위원회는 위험평가를 실시하는 역할을 담당합니다. 위험평가는 위험요소를 식별하고 분석하여 위험의 정도를 판단하는 과정을 말합니다. 이를 통해 위험을 최소화하고 안전한 환경을 조성하기 위한 대책을 마련할 수 있습니다. 따라서 위험평가위원회는 위험평가를 실시하는 것이 중요한 역할이며, 이를 통해 사회적 안전을 보장하는 역할을 수행합니다.


In [ ]:
print(f'참조 문서 : {[doc.page_content for doc in result["source_documents"]]}')

참조 문서 : ['(이하 "위험평가위원회"라 한다)를 둔다', '(이하 "위험평가위원회"라 한다)를 둔다', '(이하 "위험평가위원회"라 한다)를 둔다']


In [ ]:
llm = OpenAI(temperature=0.2, openai_api_key=api_key)
print(llm.invoke(query))



위험평가는 어떤 위험이 존재하는지 파악하고, 그 위험의 정도를 평가하여 적절한 대응책을 마련하는 과정이다. 일반적으로 다음과 같은 단계로 이루어진다.

1. 위험 식별: 조직 또는 시스템 내에서 발생할 수 있는 위험을 식별한다. 이를 위해 조직 내부의 프로세스, 시스템, 인력, 재무 등을 분석하고, 외부 환경의 변화나 잠재적 위협을 고려한다.

2. 위험 분석: 식별된 위험의 정도를 분석하여 위험의 심각성과 가능성을 평가한다. 이를 위해 각 위험에 대한 발생 가능성,


In [ ]:
query = "병해충 발생예보의 단계에서 '주의"
result = qa_chain.invoke({'query':query})
print(result['result'])


주의 단계는 발생규모, 확산속도 및 피해정도 등이 조금 더 심각한 상황을 의미합니다. 이 단계에서는 병해충의 확산을 막기 위해 적극적인 대처가 필요합니다.


벡터DB 데이터 검색

In [ ]:
# 벡터 데이터베이스 검색
documents = '''제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.
1. 대상 산림병해충의 외래병해충 여부
2. 대상 산림병해충의 생리ㆍ생태적 특성
3. 대상 산림병해충으로 인한 예상 피해 정도
4. 긴급방제 추진의 필요성과 방제방법
② 제1항에 의한 산림병해충의 위험등급은 별표 1의 산림병해충 위험평가표를 기준으로 계량화된 점수를 산정
하고 별표 2의 산림병해충 종합위험도 판정기준에 따라 다음 각 호와 같이 판정한다.
1. 종합평가점수 ‘높음’ : ‘고위험 병해충’
2. 종합평가점수 ‘중간’ : ‘중위험 병해충’
3. 종합평가점수 ‘낮음’ : ‘저위험 병해충’
4. 산림과 생활환경 및 경제에 미치는 영향에 대한 위험요소 항목 모두가 ‘가장 높은 점수’로 평가되는 경우 다른
평가 항목의 평가점수가 낮아도 ‘고위험 병해충’으로 판정할 수 있다.
5. 산림과 생활환경 및 경제에 미치는 영향에 대한 위험요소 항목 모두가 ‘가장 낮은 점수’로 평가되는 경우 다른
평가 항목의 평가점수가 높아도 ‘저위험 병해충’으로 판정할 수 있다.
6. 병해충의 정보가 부족하여 평가가 제한적인 경우 종합위험도 판정을 유예하되, 산림과 생활환경 및 경제에 미
치는 영향에 대한 위험성이 예상된다면 위험관리방안 수준을 고려하여 종합위험도를 판정할 수 있다.
③ 기존에 평가된 병해충에 대하여 필요 시 재평가를 실시할 수 있다.
④ 위원장은 제4조에서 규정한 예찰조사 결과를 위험평가에 반영할 수 있다.
'''
with open('documents.txt','w') as f:
  f.write(documents)

def create_and_search_vectorstore(texts):
  embedding = OpenAIEmbeddings(api_key=api_key)
  vectorstore = Chroma.from_documents(texts, embedding, persist_directory='./db10')
  return vectorstore

texts = load_and_split_text("documents.txt")
vt = create_and_search_vectorstore(texts)

query = '위험평가의 실시'
vt.similarity_search(query,k=5)

[Document(metadata={'source': 'documents.txt'}, page_content='평가 항목의 평가점수가 높아도 ‘저위험 병해충’으로 판정할 수 있다'),
 Document(metadata={'source': 'documents.txt'}, page_content='치는 영향에 대한 위험성이 예상된다면 위험관리방안 수준을 고려하여 종합위험도를 판정할 수 있다'),
 Document(metadata={'source': 'documents.txt'}, page_content='평가 항목의 평가점수가 낮아도 ‘고위험 병해충’으로 판정할 수 있다'),
 Document(metadata={'source': 'documents.txt'}, page_content='제4조의5(위험평가의 실시) ① 위험평가는 다음 각 호의 항목을 포함하여 실시하여야 한다.\n1. 대상 산림병해충의 외래병해충 여부\n2'),
 Document(metadata={'source': 'documents.txt'}, page_content='.\n③ 기존에 평가된 병해충에 대하여 필요 시 재평가를 실시할 수 있다.\n④ 위원장은 제4조에서 규정한 예찰조사 결과를 위험평가에 반영할 수 있다.')]

PDF Loader

In [ ]:
filepath = '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'
result = load_and_split_text(filepath)
result[0].page_content

'혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2호 | 1 |\n<요  약>'

OCR 기능을 이용해서 이미지내 텍스트 추출

In [ ]:
!pip install rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 39.2 MB/s eta 0:00:00


In [ ]:
! pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(filepath,extract_images=True)
pages = loader.load_and_split() # 페이지별

In [ ]:
pages[0]

Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'author': 'kmd kdy', 'moddate': '2022-07-29T09:03:16+09:00', 'pdfversion': '1.4', 'source': '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2호 | 1 |\n<요  약>▶혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신성장 기술분야에 대한 금융지원을 강화하고 있음     * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 11개 기관    ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성￮정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음￮본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의 기술·시장 동향을 분석함▶혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 2017년 이후 연평균 39.2% 지속 증가하고 있음￮ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임      * (‘

In [ ]:
! pip install pypdfium2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(filepath,extract_images=True)
pages =  loader.load_and_split()  # 페이지별

/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
!pip install python-docx
!pip install unstructured

In [ ]:
# 워드문서 로드
from langchain.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader("/content/문서1.docx")
pages = loader.load()  # 페이지 단위로 문서를 분리
pages

[Document(metadata={'source': '/content/문서1.docx'}, page_content='')]

이미지에 있는 text를 추출할때는
  - 1. 이미지를 word에 넣어서 pdf로 변환 - pdfloader
  - 2. 이미지 자체를 ocr 기능을 이용해서 텍스트를 추출

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import easyocr
import cv2
import numpy as np

In [ ]:
image = cv2.imread('/content/test.png')
reader = easyocr.Reader(['ko','en'])
results = reader.readtext(image)

In [ ]:
for result in results:
  print(result[1])

3. 정보통신 테마 희신성장 정책금움 현황 및 관련 산업 동향
(지원 현황) 정보동신 테마을 구성하는 기술분야별 정책금웅 지원 현화 분석결과, 공급접유움 관점
예서는 차세대무선동신미디어 분야에 가장 많은 정책지금이 투입 되고 있으며 공급광 증가율 관점
예서는 능동형컴퓨팅 분야로의 정책지금 지원 증가 속도가 가장 빠른 추세임
차세대무선통신미디어란 전습숙도 향상 소모전력 절감, 고속이동 중 금김없는 통신 등 새로운 무선
환경에 피요한 동신  인프라 및 서비스 기숙울 동징하여   4G/5G/6G
사물인터넷 .방송동신인프라
등의 품목으로 구성님
정보통신
테마 내 허신성장 성책금움 공급 규모의 약 50*름 점유하고 있으머  이는 초연결 미래
사회들 구축하기 위해 네트위크 기반 기순 사업화에 대한 정책지금 공급이 꾸준함에 따른 것으로
분석되
능동형컴퓨팅이관   거대하고 복잡해지는 데이터의 효율적 가공과 관리루
위한 인간두뇌와 유사한
형태의 정보처리기술올 말히다 인공지능 상황인지컴유팅 등의 문목으로 구성팀
컴퓨팅 기소울 활용하 다양한 사업화기 활발히 진행되고 있어 역신성장 정책금용 공급 규모가 매년
약 10093 수준오로 증가하고 있으머; 새정부의 '미래 먹거리신업
신성장 전락추진" 예 따리 인공
지능 관련 기술로의 금움지원이 늘어날
선망몸
에너지 방산 우주임공 . 인공지능시) , 비이오
단소중립 대S 스미트농업은 차세대 6대 먹거리 산업으로 선정
것으로


문서를 대상으로 정확매칭 - 유사검색 - 생성형 답변
  - 메타데이터를 추가해서 문단 단위로 분할

In [ ]:
def load_and_prepare_document():
  loader = PyPDFLoader('/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf')
  documents = loader.load()  # 페이지 단위로 로드
  # 각 페이지에 메타데이터 추가
  for doc in documents:
    page_content = doc.page_content
    page_num = doc.metadata.get("page",0) + 1 # 페이지 번호 추출
    first_line =  page_content.split('\n')[0].strip() # 첫 줄 추출
    section_title = first_line if first_line.startswith("제") else f"페이지 {page_num}"
    doc.metadata.update({"page":page_num, "section":section_title})
  # 문서 분할
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20,
                    separators=['.','\n',' ',''],keep_separator=True # 구분자를 청크 끝에 포함
                                                 )
  return text_splitter.split_documents(documents)
# 청크로 데이터베이스 생성
def create_and_search_vectorstore(texts):
  ''' chroma 벡터 db 생성, 질문에 대한 유사 문장 검색
  '''
  embedding = OpenAIEmbeddings(api_key=api_key)
  vectorstore = Chroma.from_documents(texts, embedding, persist_directory='./chroma_db')
  return vectorstore
# 유사 문장 검색 테스트 함수
def test_simularity_search(vectorstore,question = '병해충 예찰 및 발생조사 에 대해서?'):
  results = vectorstore.similarity_search(question,k=3)
  if results:
    for i, doc in enumerate(results,1):
      print(f"{i} 페이지 {doc.metadata['page']}  {doc.metadata['section']} : {doc.page_content}")
  else:
    print('유사문장 없음')

In [ ]:
texts = load_and_prepare_document()
vectorstore = create_and_search_vectorstore(texts)
test_simularity_search(vectorstore)

1 페이지 2  페이지 2 : .
 
 제2장 병해충 예찰 및 발생조사
 
제4조(예찰조사) ① 국립산림과학원장은 병해충 발생예보 발령 및 발생전망 등을 판단하기 위하여 매년 고정조사구
, 상습발생지 및 선단지 등을 대상으로 예찰조사를 실시하고 다음 각 호에 해당하는 경우에는 산림병해충위험평
2 페이지 23  페이지 23 : . 점검내용
가. 피해지의 예찰 및 진단 결과
나. 방제작업 시행의 적정성
다. 산림병해충 발생에 따른 조치 및 방제현황
5. 점검결과 조치
가. 점검 결과를 해당 기관에 통보
나
3 페이지 7  페이지 7 : .
 
제7조(발생조사) ① 예찰ㆍ방제기관의 장은 별표 3의 병해충별 발생조사 시기에 별표 4의1 발생밀도 조사요령(별표
4의1 이외 병해충은 별표 4의2의 발생밀도 조사요령에 따라 조사)에 따라 병해충별ㆍ지역별ㆍ피해도별로 관할
구역안의 병해충 발생조사를 실시하고 그 결과를 국립산림과학원장에게 매분기말 기준으로 다음 달 10일까지


In [ ]:
texts = load_and_prepare_document()
vectorstore = create_and_search_vectorstore(texts)
question = '제 4조는 어떤 내용인가'
test_simularity_search(vectorstore,question)

1 페이지 4  페이지 4 : 법제처                                                            4                                                   국가법령정보센터
산림병해충 방제규정
제4조의2(위험평가의 대상) 위험평가의 대상은 다음 각 호와 같다.
1
2 페이지 4  페이지 4 : 법제처                                                            4                                                   국가법령정보센터
산림병해충 방제규정
제4조의2(위험평가의 대상) 위험평가의 대상은 다음 각 호와 같다.
1
3 페이지 15  페이지 15 : . 그 밖에 발주기관이 요구하는 사항
② 사업시행자는 방제사업 실행 중 제출한 제1항제2호 내지 제4호의 내용변경이 있는 경우에는 변경 사항과 사
유를 감리원과 감독자를 경유하여 발주자에게 제출하여야 한다


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# OpenAI API 키 설정


# 1. PDF 문서 로드 및 준비
def load_and_prepare_documents():
    """
    PDF 문서를 로드하고, 페이지별 메타데이터를 추가해 분할.
    Returns: 분할된 텍스트 청크 리스트
    """
    print("=== PDF 문서 로드 및 준비 ===")
    try:
        loader = PyPDFLoader("/content/산림병해충 방제규정(산림청훈령)(제1664호)(20241210).pdf")
        documents = loader.load()
        print(f"로드된 문서 페이지 수: {len(documents)}")
    except Exception as e:
        print(f"PDF 로드 에러: {e}")
        return []

    # 페이지별 메타데이터 추가 (페이지 번호, 조항명 추정)
    for doc in documents:
        page_content = doc.page_content
        page_num = doc.metadata.get("page", 0) + 1
        # 조항명 추정 (예: "제1조"로 시작)
        first_line = page_content.split("\n")[0].strip()
        section_title = first_line if first_line.startswith("제") else f"페이지 {page_num}"
        doc.metadata.update({"page": page_num, "section": section_title})

    # 문서 분할 (한글 문서에 최적화)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # 로그 기반 228 청크에 적합
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""],
        keep_separator=True
    )
    texts = text_splitter.split_documents(documents)
    print(f"분할된 청크 수: {len(texts)}")
    return texts

# 2. RAG 챗봇 생성 (CoT 프롬프트)
def create_rag_chatbot(texts):
    """
    PDF 문서를 기반으로 CoT 프롬프트 RAG 챗봇 생성.
    Args:
        texts: 분할된 문서 청크
    Returns: RetrievalQA 체인
    """
    print("\n=== RAG 챗봇 생성 (COT 프롬프트) ===")
    try:
        # 벡터 데이터베이스 생성
        embeddings = OpenAIEmbeddings(api_key=api_key)
        vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
        print("벡터 데이터베이스 생성 완료")

        # LLM 초기화
        llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0.7, openai_api_key=api_key)

        # CoT 프롬프트
        template = """문서: {context}
질문: {question}
다음 단계를 따라 한글로 답변하세요:
1) 문서에서 질문과 관련된 문장을 검색합니다.
2) 관련 문장이 있으면 해당 항목들을 나열한다
3) 문장에 없으면 "문서에 정보 없음"을 명시하고, 일반 지식으로 간결히 답변합니다.
답변은 정확해야 하며, 조항명과 페이지 번호를 명확히 표시하세요."""
        prompt_template = PromptTemplate(input_variables=["context", "question"], template=template)

        # RetrievalQA 체인
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt_template}
        )
        return qa_chain
    except Exception as e:
        print(f"RAG 챗봇 생성 에러: {e}")
        return None

# 3. 챗봇 테스트
def test_chatbot(qa_chain):
    """
    CoT 프롬프트로 챗봇 테스트.
    Args:
        qa_chain: RetrievalQA 체인
    """
    print("\n=== 챗봇 테스트 ===")
    questions = [
        "산림병해충 방제의 원칙은 무엇인가?",
        "방제 절차는 무엇인가?",
        "산림병해충 예방 방법은 무엇인가?",
        "제2조에서 정의된 용어는 무엇인가?",
        "항공방제란 무엇인가?",
        "위험평가의 대상은"
    ]
    for q in questions:
        print(f"\n질문: {q}")
        try:
            result = qa_chain.invoke({"query": q})
            print(f"COT 답변: {result['result']}")
            # print(f"참조 문서: {[f'페이지 {doc.metadata['page']} ({doc.metadata['section']}): {doc.page_content}' for doc in result['source_documents']]}")
        except Exception as e:
            print(f"COT 에러: {e}")
        print("-" * 50)

# 메인 실행
def main():
    # 문서 로드 및 분할
    texts = load_and_prepare_documents()
    if not texts:
        print("문서 처리 실패, 종료.")
        return

    # CoT 기반 챗봇 생성
    qa_chain = create_rag_chatbot(texts)
    if not qa_chain:
        print("챗봇 생성 실패, 종료.")
        return

    # 챗봇 테스트
    test_chatbot(qa_chain)

if __name__ == "__main__":
    main()

=== PDF 문서 로드 및 준비 ===
로드된 문서 페이지 수: 28
분할된 청크 수: 228

=== RAG 챗봇 생성 (COT 프롬프트) ===
벡터 데이터베이스 생성 완료

=== 챗봇 테스트 ===

질문: 산림병해충 방제의 원칙은 무엇인가?
COT 답변: 1) 문서에서 질문과 관련된 문장을 검색합니다.

2) 관련 문장  
- "발생한 산림병해충에 효과적이며 적용가능한 약제를 선택하고, 「산림병해충 방제 농약등의 안전사용지침」을 준수할 것"  
- "제11조(조사의 기본 원칙) ① 방제 대상목 조사는 전수조사를 원칙으로 한다. 다만, 다음 각 호의 경우에는 표준지(조사, 측정, 평가 등의 기준이 되는 지역)조사를 할 수 있다."

3) 답변  
산림병해충 방제의 원칙은 다음과 같습니다.  
- 발생한 산림병해충에 효과적이며 적용 가능한 약제를 선택하고, 「산림병해충 방제 농약등의 안전사용지침」을 준수해야 합니다.  
- 방제 대상목 조사는 전수조사를 원칙으로 하되, 예외적으로 표준지 조사를 할 수 있습니다.  
(출처: 제11조(조사의 기본 원칙), 문서 첫 페이지)
--------------------------------------------------

질문: 방제 절차는 무엇인가?
COT 답변: 1) 문서에서 질문과 관련된 문장을 검색합니다.

2) 관련 문장이 있으면 해당 항목들을 나열한다

문서에서 "방제 절차"와 관련된 것으로 보이는 문장은 다음과 같습니다:

1. 방제명령을 받은 자가 방제명령을 이행하지 않거나 소홀히 하는 경우  
2. 산림소유자의 소재파악이 불가능한 경우  
3. 산림소유자가 방제능력이 없다고 판단될 경우  
4. 다른 지역으로 확산될 우려가 있어 긴급히 방제가 필요한 경우  
5. 기타 직접방제를 하는 것이 타당하다고 인정되는 경우

3) 문장에 없으면 "문서에 정보 없음"을 명시하고, 일반 지식으로 간결히 답변합니다.

따라서, 문서에 명시된 방제 절차 관련 조항은 아래와 같습니다.

---

**[조항명, 페이

RAG - QA 구축

In [ ]:
# pdf 파일 로드
loader = PyPDFLoader('/content/대한민국헌법(헌법)(제00010호)(19880225).pdf')
pages = loader.load_and_split()

In [ ]:
# 청크로 분할 1000 - size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs =  text_splitter.split_documents(pages)

In [ ]:
# 임베딩
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = Chroma.from_documents(docs, embeddings)
retriver = vectorstore.as_retriever()

In [ ]:
# 프롬프트 모델
llm = ChatOpenAI(openai_api_key=api_key, model = 'gpt-4o-mini')

In [ ]:
# 프롬프트 엔지니어링
prompt = '''
당신은 질문 답변 어시스턴트 입니다.
제공된 문맥을 이용해서 질문에 답하세요
문맥에 답변이 없으면 "정보 없음"이라고 표시하고 일반 지식으로 간결히 답변하세요
답변은 최대 세 문장으로 작성하세요

문맥 : {context}
질문 : {question}
'''

prompt_template = PromptTemplate(input_variables=["context", "question"], template=prompt)

# RetrievalQA 체인
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)
result = qa_chain.invoke('대통령의 의무')
print(f"결과 : {result['result']}")
print("참조문서 : ")
for doc in result['source_documents']:
  print(doc.page_content)

결과 : 대통령의 주요 의무는 직책을 성실히 수행하는 것이며, 또한 헌법과 법률에 따라 국군을 통수하고 국가안위에 관한 중요정책을 국민투표에 붙일 수 있는 권한이 있습니다. 대통령은 필요한 경우 긴급한 조치를 취하여 국가의 안전보장이나 공공의 안녕질서를 유지해야 합니다. 추가로, 조약을 체결하고 외교사절을 파견하는 등의 외교적 책임도 포함됩니다.
참조문서 : 
노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다.”
 
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의
안녕질서를 유지하기 위하여 긴급한 조치가 필요하고 국회의 집회를 기다릴 여유가 없을 때에 한하여 최소한으로
필요한 재정ㆍ경제상의 처분을 하거나 이에 관하여 법률의 효력을 가지는 명령을 발할 수 있다.
노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다.”
 
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방

RAG-memory
  - 페이지별 메타에이터 활용
  - 대화 메모리 포함 세션별 기록 저장

In [221]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
# 위의 코드를 재 사용하면서
# 청크로 분할 1000 - size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs =  text_splitter.split_documents(pages)
# 임베딩
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = Chroma.from_documents(docs, embeddings)
retriver = vectorstore.as_retriever()

# 채팅 히스토리와 사용지 질문 문맥을 고려해서 질문 재 구성
context_prompt = ''' 대화 기록과 최신 사용자 질문을 바탕으로, \
대화 기록 없이도 이해할 수 있는 독립적인 질문을 구성하세요. \
질문에 답변하지 말고, 필요한 경우 질문만 재구성하거나 그대로 반환하세요
'''
contextualize_prompt =  ChatPromptTemplate.from_messages([
 ("system", context_prompt)   ,
 MessagesPlaceholder(variable_name="chat_history"),
 ("human", "{input}")
])
# 대화 기록을 고려한 검색기 생성
# llm = ChatOpenAI(model = 'gpt-4o-mini')
llm = ChatOpenAI(openai_api_key=api_key)
history_aware_retriever =  create_history_aware_retriever(llm, retriver,contextualize_prompt)
# 대화 기록 example
chat_history = [
    HumanMessage(content="대통령의 임기는 몇년이야?"),
    AIMessage(content="대통령의 임기는 5년 입니다.")
]
result = contextualize_prompt.invoke({"input" : "국회의원의 임기는??", "chat_history":chat_history})

In [218]:
result

[Document(metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'source': '/content/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'creator': 'PyPDF', 'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 14, 'page_label': '5', 'page': 4}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.\n③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 

In [225]:
result = history_aware_retriever.invoke({"input" : "국회의원의 임기는??", "chat_history":chat_history})
print(result[0].page_content)

법제처                                                            5                                                       국가법령정보센터
대한민국헌법
③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.
 
제42조 국회의원의 임기는 4년으로 한다.
 
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.
②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.
 
제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.
 
제46조 ①국회의원은 청렴의 의무가 있다.
②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.
③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.


In [ ]:
# RAG : 문서기반 검색시스템
  # 벡터DB
# LangChain : RAG데이터를 LLM으로 전달해서 생성형 ai 실행
  # 프롬프트 엔지니어링

# 다양한 형태의 문서를 load
# json, docx, pdf, csv , txt, ocr(image 텍스트 추출), notion
# 문서기반으로 요약및 QA 등등....
# 해당 내용이 없으면 일반 gpt로 추론( 일반모델이 아니라 특화된 파인튜닝 모델도 가능)